### Polygonal ROI

* This code has been taken from: https://stackoverflow.com/questions/37099262/drawing-filled-polygon-using-mouse-events-in-open-cv-using-python/37235130. 

* I have made the following modifications to the code:

    *   Instead of taking a fixed size canvas, I have modifed it to take an RGB image.
    *   Color of the lines being drawn and the final lines being drawn has been changed.
    *   The code now saves roi masked image on the disk.


* Made use of one of my own utility `put_text_with_background` function which is kept in `helper.py` file

In [5]:
import numpy as np
import cv2
from helper import put_text_with_background

In [6]:
class PolygonDrawer:
    def __init__(self, window_name, image):
        self.FINAL_LINE_COLOR = (0, 255, 0)
        self.WORKING_LINE_COLOR = (0, 0, 255)
        self.image = image
        self.window_name = window_name
        self.done = False
        self.current = (0, 0)
        self.points = []
        self.instructions = [
            "Use mouse to select an ROI",
            " ",
            "Press 'Esc' to finish adding points",
            " ",
            "After that press any key to confirm",
        ]

    def on_mouse(self, event, x, y, buttons, user_param):
        """
        Mouse callback function that handles mouse events for polygon drawing.
        """
        if self.done:
            return
        if event == cv2.EVENT_MOUSEMOVE:
            self.current = (x, y)
        elif event == cv2.EVENT_LBUTTONDOWN:
            self.add_point(x, y)
        elif event == cv2.EVENT_RBUTTONDOWN:
            self.complete_polygon()

    def add_point(self, x, y):
        """
        Adds a point to the list of polygon points.
        """
        print(f"Adding point #{len(self.points)} with position ({x}, {y})")
        self.points.append((x, y))

    def complete_polygon(self):
        """
        Marks polygon drawing as complete.
        """
        print(f"Completing polygon with {len(self.points)} points.")
        self.done = True

    def draw_polygon(self, canvas):
        """
        Draws the current state of the polygon on the canvas.
        """
        if len(self.points) > 0:
            cv2.polylines(
                canvas,
                np.array([self.points]),
                False,
                self.FINAL_LINE_COLOR,
                thickness=4,
            )
            if self.current != (0, 0):
                cv2.line(
                    canvas,
                    self.points[-1],
                    self.current,
                    self.WORKING_LINE_COLOR,
                    thickness=4,
                )
        return canvas

    def run(self):
        """
        Runs the polygon drawing application.
        """
        cv2.namedWindow(self.window_name, flags=cv2.WINDOW_KEEPRATIO)
        cv2.imshow(self.window_name, self.image)
        cv2.waitKey(1)
        cv2.setMouseCallback(self.window_name, self.on_mouse)

        while not self.done:
            canvas = self.image.copy()
            canvas = put_text_with_background(canvas, self.instructions, position="top-right")
            canvas = self.draw_polygon(canvas)
            cv2.imshow(self.window_name, canvas)
            if cv2.waitKey(50) == 27:  # ESC hit
                self.done = True

        mask = np.zeros(self.image.shape[:2], np.uint8)
        if len(self.points) > 0:
            cv2.fillPoly(mask, np.array([self.points]), 255)
            self.image = cv2.bitwise_and(self.image, self.image, mask=mask)

        cv2.imshow(self.window_name, self.image)
        cv2.waitKey()
        cv2.destroyAllWindows()
        return self.image

In [7]:
def create_ROI(image, write_roi=False):
    print("====> Use left click to draw polygon, right click to release and finish")
    print()
    pd = PolygonDrawer(window_name="Draw Polygonal ROI", image=image)
    result_image = pd.run()
    if write_roi:
        cv2.imwrite("polygon-roi.jpg", result_image)
    print("Polygon Points:", pd.points)
    if not pd.points:
        return None
    return pd.points

In [8]:
if __name__ == "__main__":
    image = cv2.imread('/home/acer/workspace/images/output_images/1/image_0005.jpg')
    create_ROI(image)

====> Use left click to draw polygon, right click to release and finish

Adding point #0 with position (424, 232)
Adding point #1 with position (940, 118)
Adding point #2 with position (1405, 246)
Adding point #3 with position (1463, 567)
Adding point #4 with position (926, 706)
Adding point #5 with position (570, 567)
Adding point #6 with position (478, 771)
Adding point #7 with position (235, 761)
Adding point #8 with position (555, 929)
Adding point #9 with position (1194, 927)
Adding point #10 with position (1712, 770)
Adding point #11 with position (1720, 426)
Polygon Points: [(424, 232), (940, 118), (1405, 246), (1463, 567), (926, 706), (570, 567), (478, 771), (235, 761), (555, 929), (1194, 927), (1712, 770), (1720, 426)]
